In [ ]:
trn_trans=pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
trn=pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
tst_trans=pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')
tst=pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')
fnl_trn=trn.merge(trn_trans,on='TransactionID',how='left')
fnl_tst=tst.merge(tst_trans,on='TransactionID',how='left')
fnl_trn=fnl_trn.drop(list(fnl_trn.filter(regex='V')),1)
fnl_tst=fnl_tst.drop(list(fnl_tst.filter(regex='V')),1)
import gc
dk={}
for i in range(1,39):
    if i<10:
        dk['id-0'+str(i)]='id_0'+str(i)
    else:
        dk['id-'+str(i)]='id_'+str(i)
fnl_tst=fnl_tst.rename(columns=dk)
del([trn,tst,trn_trans,tst_trans])
gc.collect()

In [ ]:
fnl_trn['is_train']=1
fnl_tst['is_train']=0
fnl=pd.concat([fnl_trn,fnl_tst],0)
fnl=fnl.reset_index(drop=True)
gc.collect()
numerical = ["TransactionAmt", "dist1", "dist2"] + ["C" + str(i) for i in range(1, 15)] + \
            ["D" + str(i) for i in range(1, 16)] 
categorical = ["ProductCD", "card1", "card2", "card3", "card4", "card5", "card6", "addr1", "addr2",
               "P_emaildomain", "R_emaildomain",
              "DeviceInfo", "DeviceType"] + ["id_0" + str(i) for i in range(1, 10)] +\
                ["id_" + str(i) for i in range(10, 39)] + \
                 ["M" + str(i) for i in range(1, 10)]
for col in numerical:
    fnl[col]=fnl[col].fillna(fnl[col].mean())
for col in categorical:
    fnl[col]=fnl[col].fillna('nan')

fnl['days']=fnl['TransactionDT']//86400
fnl['id']=fnl['days']-fnl['D1']
fnl['id']=fnl['id'].astype(str)+fnl['id'].astype(str)+fnl['card1'].astype(str)+fnl['addr1'].astype(str)+fnl['ProductCD'].astype(str)
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
fnl['id']=le.fit_transform(fnl['id'])
from tqdm import tqdm
for col in tqdm(numerical):
    try:
        fnl[col+'_mean']=fnl.groupby(['id'])[col].transform('mean')
        fnl[col+'_std']=fnl.groupby(['id'])[col].transform('std')
    except:
        continue
fnl=fnl.drop(['id'],1)
from tqdm import tqdm
for col in tqdm(categorical):
        print('before '+str(fnl[col].nunique()))
        fnl.loc[~fnl[col].isin(list(fnl[col].value_counts().index[:5])),col]='other'
        print('before '+str(fnl[col].nunique()))
        if col!='card1' or col !='id_02':
            fnl_dum=pd.get_dummies(fnl[col]).astype('uint8')
            fnl_dum.columns=list(np.char.add(col,np.asarray(list(fnl_dum)).astype(str)))
            fnl_dum.columns=fnl_dum.columns+'_dum'
            
            #fnl=fnl.drop([col],1)
            fnl=pd.concat([fnl,fnl_dum],1)
fnl['day']=fnl['TransactionDT']//86400
fnl['month']=fnl['day']//30
fnl['week']=fnl['day']//7

In [ ]:
import gc
fnl.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in fnl.columns]
fnl=fnl.drop(['TransactionID','TransactionDT'],1)
fnl=fnl.reset_index(drop=True)
gc.collect()
fnl=fnl.fillna(-1)

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        df[col]=df[col].fillna(-1)
        df[col]=df[col].replace([np.inf,-np.inf],-1)
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


In [ ]:
ls=set(list(fnl))
tot=['ProductCD','card1','card2','DeviceType','DeviceInfo','card3','card4','card5','card6','P_emaildomain','R_emaildomain','day','month','week','is_train','isFraud']
ls=list(fnl.filter(regex='dum'))
tot+=ls
ls=list(fnl.select_dtypes(include=object))
tot+=ls
tot=fnl[tot]
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
ls=list(fnl.drop(tot,1))
fnl=pd.DataFrame(ss.fit_transform(fnl.drop(tot,1)))
fnl.columns=ls
fnl=reduce_mem_usage(fnl)

In [ ]:
tst=pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv',usecols=['TransactionDT'])
trn=pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv',usecols=['TransactionDT'])
sdo1=pd.concat([trn,tst],0)
sdo1['day']=sdo1['TransactionDT']//86400
sdo1['week']=sdo1['day']//7
sdo1['month']=sdo1['day']//30
sdo1=sdo1.reset_index(drop=True)
fnl=pd.concat([fnl,sdo1.drop(['TransactionDT'],1),tot],1)
del([sdo1,trn,tst,tot])
gc.collect()

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv
trn_trans=pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
trn=pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
tst_trans=pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')
tst=pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')
fnl_trn=trn.merge(trn_trans,on='TransactionID',how='left')
fnl_tst=tst.merge(tst_trans,on='TransactionID',how='left')
fnl_trn=fnl_trn.drop(list(fnl_trn.filter(regex='V')),1)
fnl_tst=fnl_tst.drop(list(fnl_tst.filter(regex='V')),1)
import gc
dk={}
for i in range(1,39):
    if i<10:
        dk['id-0'+str(i)]='id_0'+str(i)
    else:
        dk['id-'+str(i)]='id_'+str(i)
fnl_tst=fnl_tst.rename(columns=dk)
del([trn,tst,trn_trans,tst_trans])
gc.collect()
0
fnl_trn['is_train']=1
fnl_tst['is_train']=0
fnl=pd.concat([fnl_trn,fnl_tst],0)
fnl=fnl.reset_index(drop=True)
gc.collect()
numerical = ["TransactionAmt", "dist1", "dist2"] + ["C" + str(i) for i in range(1, 15)] + \
            ["D" + str(i) for i in range(1, 16)] 
categorical = ["ProductCD", "card1", "card2", "card3", "card4", "card5", "card6", "addr1", "addr2",
               "P_emaildomain", "R_emaildomain",
              "DeviceInfo", "DeviceType"] + ["id_0" + str(i) for i in range(1, 10)] +\
                ["id_" + str(i) for i in range(10, 39)] + \
                 ["M" + str(i) for i in range(1, 10)]
for col in numerical:
    fnl[col]=fnl[col].fillna(fnl[col].mean())
for col in categorical:
    fnl[col]=fnl[col].fillna('nan')

fnl['days']=fnl['TransactionDT']//86400
fnl['id']=fnl['days']-fnl['D1']
fnl['id']=fnl['id'].astype(str)+fnl['id'].astype(str)+fnl['card1'].astype(str)+fnl['addr1'].astype(str)+fnl['ProductCD'].astype(str)
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
fnl['id']=le.fit_transform(fnl['id'])
from tqdm import tqdm
for col in tqdm(numerical):
    try:
        fnl[col+'_mean']=fnl.groupby(['id'])[col].transform('mean')
        fnl[col+'_std']=fnl.groupby(['id'])[col].transform('std')
    except:
        continue
fnl=fnl.drop(['id'],1)
from tqdm import tqdm
for col in tqdm(categorical):
        print('before '+str(fnl[col].nunique()))
        fnl.loc[~fnl[col].isin(list(fnl[col].value_counts().index[:5])),col]='other'
        print('before '+str(fnl[col].nunique()))
        if col!='card1' or col !='id_02':
            fnl_dum=pd.get_dummies(fnl[col]).astype('uint8')
            fnl_dum.columns=list(np.char.add(col,np.asarray(list(fnl_dum)).astype(str)))
            fnl_dum.columns=fnl_dum.columns+'_dum'
            
            #fnl=fnl.drop([col],1)
            fnl=pd.concat([fnl,fnl_dum],1)
fnl['day']=fnl['TransactionDT']//86400
fnl['month']=fnl['day']//30
fnl['week']=fnl['day']//7
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until
100%|██████████| 32/32 [00:40<00:00,  1.27s/it]
  0%|          | 0/60 [00:00<?, ?it/s]
before 5
before 5
  2%|▏         | 1/60 [00:02<02:33,  2.60s/it]
before 17091
before 6
  3%|▎         | 2/60 [00:10<04:01,  4.16s/it]
before 502
before 6
  5%|▌         | 3/60 [00:16<04:30,  4.75s/it]
before 134
before 6
  7%|▋         | 4/60 [00:22<04:47,  5.13s/it]
before 5
before 5
  8%|▊         | 5/60 [00:28<04:50,  5.28s/it]
before 139
before 6
 10%|█         | 6/60 [00:34<04:54,  5.45s/it]
before 5
before 5
 12%|█▏        | 7/60 [00:39<04:49,  5.46s/it]
before 442
before 6
 13%|█▎        | 8/60 [00:45<04:50,  5.59s/it]
before 94
before 6
 15%|█▌        | 9/60 [00:51<04:53,  5.76s/it]
before 61
before 6
 17%|█▋        | 10/60 [00:57<04:46,  5.74s/it]
before 61
before 6
 18%|█▊        | 11/60 [01:02<04:38,  5.67s/it]
before 2800
before 6
 20%|██        | 12/60 [01:08<04:29,  5.61s/it]
before 3
before 3
 22%|██▏       | 13/60 [01:13<04:21,  5.57s/it]
before 90
before 6
 23%|██▎       | 14/60 [01:19<04:19,  5.63s/it]
before 198052
before 6
 25%|██▌       | 15/60 [01:25<04:18,  5.74s/it]
before 26
before 6
 27%|██▋       | 16/60 [01:31<04:12,  5.75s/it]
before 17
before 6
 28%|██▊       | 17/60 [01:36<04:07,  5.75s/it]
before 98
before 6
 30%|███       | 18/60 [01:42<04:02,  5.77s/it]
before 102
before 6
 32%|███▏      | 19/60 [01:48<03:57,  5.78s/it]
before 93
before 6
 33%|███▎      | 20/60 [01:54<03:49,  5.74s/it]
before 98
before 6
 35%|███▌      | 21/60 [01:59<03:43,  5.74s/it]
before 52
before 6
 37%|███▋      | 22/60 [02:05<03:40,  5.79s/it]
before 68
before 6
 38%|███▊      | 23/60 [02:11<03:34,  5.79s/it]
before 413
before 6
 40%|████      | 24/60 [02:17<03:30,  5.84s/it]
before 3
before 3
 42%|████▏     | 25/60 [02:23<03:21,  5.75s/it]
before 56
before 6
 43%|████▎     | 26/60 [02:29<03:17,  5.82s/it]
before 29
before 6
 45%|████▌     | 27/60 [02:34<03:12,  5.83s/it]
before 4
before 4
 47%|████▋     | 28/60 [02:40<03:04,  5.76s/it]
before 3
before 3
 48%|████▊     | 29/60 [02:46<02:57,  5.72s/it]
before 128
before 6
 50%|█████     | 30/60 [02:52<02:54,  5.80s/it]
before 20
before 6
 52%|█████▏    | 31/60 [02:58<02:48,  5.83s/it]
before 569
before 6
 53%|█████▎    | 32/60 [03:04<02:44,  5.87s/it]
before 548
before 6
 55%|█████▌    | 33/60 [03:09<02:38,  5.87s/it]
before 735
before 6
 57%|█████▋    | 34/60 [03:15<02:32,  5.85s/it]
before 36
before 6
 58%|█████▊    | 35/60 [03:21<02:25,  5.81s/it]
before 4
before 4
 60%|██████    | 36/60 [03:27<02:18,  5.75s/it]
before 18
before 6
 62%|██████▏   | 37/60 [03:32<02:12,  5.76s/it]
before 441
before 6
 63%|██████▎   | 38/60 [03:38<02:06,  5.76s/it]
before 116
before 6
 65%|██████▌   | 39/60 [03:44<02:01,  5.77s/it]
before 3
before 3
 67%|██████▋   | 40/60 [03:50<01:54,  5.73s/it]
before 3
before 3
 68%|██████▊   | 41/60 [03:55<01:48,  5.70s/it]
before 3
before 3
 70%|███████   | 42/60 [04:01<01:42,  5.71s/it]
before 88
before 6
 72%|███████▏  | 43/60 [04:07<01:37,  5.72s/it]
before 173
before 6
 73%|███████▎  | 44/60 [04:12<01:31,  5.71s/it]
before 7
before 6
 75%|███████▌  | 45/60 [04:18<01:26,  5.79s/it]
before 462
before 6
 77%|███████▋  | 46/60 [04:24<01:20,  5.77s/it]
before 5
before 5
 78%|███████▊  | 47/60 [04:30<01:14,  5.76s/it]
before 3
before 3
 80%|████████  | 48/60 [04:35<01:08,  5.73s/it]
before 3
before 3
 82%|████████▏ | 49/60 [04:41<01:02,  5.72s/it]
before 3
before 3
 83%|████████▎ | 50/60 [04:47<00:57,  5.72s/it]
before 3
before 3
 85%|████████▌ | 51/60 [04:53<00:51,  5.73s/it]
before 3
before 3
 87%|████████▋ | 52/60 [04:58<00:45,  5.74s/it]
before 3
before 3
 88%|████████▊ | 53/60 [05:04<00:40,  5.73s/it]
before 3
before 3
 90%|█████████ | 54/60 [05:10<00:34,  5.70s/it]
before 4
before 4
 92%|█████████▏| 55/60 [05:16<00:28,  5.74s/it]
before 3
before 3
 93%|█████████▎| 56/60 [05:21<00:23,  5.77s/it]
before 3
before 3
 95%|█████████▌| 57/60 [05:27<00:17,  5.78s/it]
before 3
before 3
 97%|█████████▋| 58/60 [05:33<00:11,  5.76s/it]
before 3
before 3
 98%|█████████▊| 59/60 [05:39<00:05,  5.74s/it]
before 3
before 3
100%|██████████| 60/60 [05:44<00:00,  5.75s/it]
z=fnl.isna().sum()
z[z>0]
isFraud               506691
TransactionAmt_std    245596
dist1_std             245596
dist2_std             245596
C1_std                245596
C2_std                245596
C3_std                245596
C4_std                245596
C5_std                245596
C6_std                245596
C7_std                245596
C8_std                245596
C9_std                245596
C10_std               245596
C11_std               245596
C12_std               245596
C13_std               245596
C14_std               245596
D1_std                245596
D2_std                245596
D3_std                245596
D4_std                245596
D5_std                245596
D6_std                245596
D7_std                245596
D8_std                245596
D9_std                245596
D10_std               245596
D11_std               245596
D12_std               245596
D13_std               245596
D14_std               245596
D15_std               245596
dtype: int64
import lightgbm as lgb
import gc
fnl.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in fnl.columns]
fnl=fnl.drop(['TransactionID','TransactionDT'],1)
fnl=fnl.reset_index(drop=True)
gc.collect()
fnl=fnl.fillna(-1)
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        df[col]=df[col].fillna(-1)
        df[col]=df[col].replace([np.inf,-np.inf],-1)
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
kps=['id_20nan',
 'id_31nan',
 'id_17nan',
 'id_19nan',
 'D8_mean',
 'D5',
 'id_06nan',
 'id_05nan',
 'D9_mean',
 'id_02nan',
 'R_emaildomainnan',
 'id_01nan',
 'id_12nan',
 'id_11100_0',
 'id_10nan',
 'id_09nan',
 'D11_mean',
 'C9',
 'id_03nan',
 'id_04nan',
 'C5_mean',
 'M1nan',
 'M3nan',
 'M2nan',
 'D6_mean',
 'D8_std',
 'C12',
 'C8',
 'D8',
 'C10_mean',
 'D14',
 'D7_mean',
 'C12_mean',
 'D9_std',
 'D2_mean',
 'D13_mean',
 'C8_mean',
 'D9',
 'C4_mean',
 'C9_mean',
 'C5_std',
 'D3',
 'id_17225_0',
 'C2',
 'DeviceTypenan',
 'C7',
 'D11_std',
 'id_37T',
 'id_02other',
 'id_35F',
 'dist2_mean',
 'id_090_0',
 'card3185_0',
 'dist1_mean',
 'id_38nan',
 'id_15nan',
 'id_35nan',
 'id_36nan',
 'id_37nan',
 'D1_std',
 'D7',
 'card3150_0',
 'D1_mean',
 'dist2_std',
 'D10_std',
 'C7_mean',
 'C4',
 'ProductCDW',
 'C12_std',
 'C8_std',
 'D2',
 'D12_std',
 'C10_std',
 'addr287_0',
 'id_11nan',
 'id_29nan',
 'id_28nan',
 'C4_std',
 'D12_mean',
 'M3T',
 'M6nan',
 'M8nan',
 'M9nan',
 'C7_std',
 'C1',
 'id_36F',
 'C10',
 'M4nan',
 'C11_mean',
 'ProductCDC',
 'D14_mean',
 'D15_mean',
 'D7_std',
 'D6_std',
 'id_20other',
 'D14_std',
 'D12',
 'D10_mean',
 'dist1',
 'M1T',
 'DeviceInfonan',
 'id_28Found',
 'dist1_std',
 'id_29Found',
 'M7nan',
 'C1_mean',
 'id_31other',
 'C2_mean',
 'C5','day','month','week',
 'id_13nan',
 'D13_std',
 'D15',
 'M2T',
 'R_emaildomaingmail_com',
 'M9T',
 'D13',
 'id_15Found',
 'id_030_0',
 'TransactionAmt',
 'C13_mean',
 'D4_mean',
 'addr1nan',
 'addr2nan',
 'id_040_0',
 'M6T',
 'DeviceTypemobile',
 'M4M2',
 'D1',
 'C13_std',
 'D6',
 'id_19other',
 'C13',
 'M7F',
 'C11_std',
 'card6credit',
 'D15_std',
 'C1_std',
 'C14_mean',
 'C6',
 'D4_std',
 'M6F',
 'D11',
 'D5_std',
 'card5other',
 'D4',
 'id_18nan',
 'C2_std',
 'card6debit',
 'addr1other',
 'C11',
 'id_38F',
 'D10',
 'id_1352_0',
 'id_16Found',
 'id_100_0',
 'C14_std',
 'id_20507_0',
 'D5_mean',
 'DeviceInfoother',
 'id_28New',
 'C9_std',
 'D3_mean',
 'id_16nan',
 'id_38T',
 'M8F',
 'TransactionAmt_mean',
 'id_01other',
 'M7_dsct',
 'C14',
 'id_29NotFound',
 'card1_std',
 'C6_std',
 'M8_dsct',
 'card2other',
 'M8T',
 'id_19266_0',
 'id_10_dsct',
 'id_01_5_0',
 'card5166_0',
 'id_12NotFound',
 'id_050_0',
 'id_08nan',
 'id_07nan',
 'id_23nan',
 'id_27nan',
 'id_22nan',
 'M3_dsct',
 'id_26nan',
 'M6_dsct',
 'id_16_dsct',
'is_train'
 'card17919',
 'M5_dsct',
 'D3_std',
 'id_37_dsct',
 'id_03_dsct',
 'id_04_dsct',
 'id_010_0',
 'card2111_0',
 'id_15New',
 'id_29_dsct',
 'id_20_dsct',
 'id_37F',
 'id_30other',
 'id_16NotFound',
 'id_1364_0',
 'M1_dsct',
 'D2_std',
 'id_09_dsct',
 'card5102_0',
 'id_051_0',
 'dist2',
 'P_emaildomaingmail_com',
 'id_24_dsct',
 'id_21nan',
 'id_13other',
 'DeviceInfoWindows',
 'id_1815_0',
 'R_emaildomainhotmail_com',
 'id_32_dsct',
 'addr287.0',
 'card3150.0',
 'card2583_0',
 'id_14_300_0',
 'R_emaildomainother',
 'card5226.0',
 'id_28_dsct',
 'P_emaildomaingmail.com',
 'P_emaildomainanonymous_com',
 'id_33nan',
 'id_06_dsct',
 'id_30nan',
 'id_031.0',
 'id_091.0',
 'id_2241.0',
 'id_06-1.0',
 'R_emaildomainyahoo.com',
 'id_11100.0',
 'id_05_dsct',
 'addr1264.0',
 'P_emaildomainanonymous.com',
 'card2555.0',
 'card2111.0',
 'addr1299.0',
 'card19500',
 'card2583.0',
 'id_19529.0',
 'id_14-480.0',
 'id_17166.0',
 'R_emaildomainanonymous.com',
 'addr1204.0',
 'addr1325.0',
 'id_1352.0',
 'id_100.0',
 'id_1327.0',
 'id_090.0',
 'id_14-360.0',
 'card3185.0',
 'id_08-33.0',
 'card6charge card',
 'id_08-100.0',
 'id_0716.0',
 'id_0714.0',
 'id_08-34.0',
 'id_092.0',
 'id_1364.0',
 'id_0712.0',
 'id_093.0',
 'id_10-1.0',
 'id_10-5.0',
 'id_10-6.0',
 'id_1349.0',
 'addr296.0',
 'id_20325.0',
 'addr260.0',
 'id_1195.08000183105469',
 'id_1195.16000366210938',
 'id_080.0',
 'id_070.0',
 'card4american express',
 'id_04-9.0',
 'id_1812.0',
 'card3117.0',
 'id_21252.0',
 'id_032.0',
 'id_033.0',
 'card3106.0',
 'id_021102.0',
 'id_04-6.0',
 'id_2243.0',
 'id_2214.0',
 'id_02696.0',
 'id_021083.0',
 'id_2233.0','isFraud'
 'id_051.0',
 'addr232.0',
 'id_052.0',
 'id_17159.0',
 'DeviceInfoTrident/7.0','day','month','week',
 'id_14-420.0',
 'id_19427.0','ProductCD','card1','card2','card3','card4','card5','card6','P_emaildomain','R_emaildomain','is_train']
fnl.shape
(1097231, 458)
ls=set(list(fnl))
kps=set(kps).intersection(ls)
# fnl=fnl[kps]
tot=['ProductCD','card1','card2','DeviceType','DeviceInfo','card3','card4','card5','card6','P_emaildomain','R_emaildomain','day','month','week','is_train','isFraud']
ls=list(fnl.filter(regex='dum'))
tot+=ls
ls=list(fnl.select_dtypes(include=object))
tot+=ls
tot=fnl[tot]
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
ls=list(fnl.drop(tot,1))
fnl=pd.DataFrame(ss.fit_transform(fnl.drop(tot,1)))
fnl.columns=ls
fnl=reduce_mem_usage(fnl)
Memory usage of dataframe is 812.01 MB
Memory usage after optimization is: 203.00 MB
Decreased by 75.0%
fnl.shape
(1097231, 97)
tst=pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv',usecols=['TransactionDT'])
trn=pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv',usecols=['TransactionDT'])
sdo1=pd.concat([trn,tst],0)
sdo1['day']=sdo1['TransactionDT']//86400
sdo1['week']=sdo1['day']//7
sdo1['month']=sdo1['day']//30
sdo1=sdo1.reset_index(drop=True)
fnl=pd.concat([fnl,sdo1.drop(['TransactionDT'],1),tot],1)
del([sdo1,trn,tst,tot])
gc.collect()
0
#fnl=fnl.drop(['isFraud'],1)
trn=pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv',usecols=['isFraud'])
fnl['isFraud']=trn['isFraud']
#fnl=pd.concat([fnl,tot],1)
fnl=fnl.loc[:,~(fnl.columns.duplicated())]
fnl.to_csv('final.csv')